## Factors

一个因子是从资产和时刻到数字的函数。

![因子函数](./factor_fun.png)

在管道中，因子是最常用的术语，表示任何计算产生数值结果的结果。因素需要一列数据以及窗口长度作为输入。

管道中最简单的因子是内置因子。内置因子是预先构建的，可执行常用计算。作为第一个例子，我们来制定一个因子，计算每个资产在10天后窗口的平均收盘价。我们可以使用`SimpleMovingAverage`内置因子计算指定窗口长度（10天）内输入数据的平均值（收盘价格）。为此，我们需要导入我们的内置`SimpleMovingAverage`因子和`USEquityPricing`数据集。

In [1]:
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage

## Creating a Factor

让我们回到上一课的`make_pipeline`函数，并实例化一个`SimpleMovingAverage`因子。 要创建`SimpleMovingAverage`因子，我们可以使用两个参数调用`SimpleMovingAverage`构造函数：`inputs`（它必须是BoundColumn对象的列表）和`window_length`（必须是一个整数，指示我们的移动平均值计算应该接收多少天的数据）。

我们稍后将更深入地讨论`BoundColumn`;现在我们只需要知道`BoundColumn`是一个对象，指示应将哪种数据传递给我们的因子。

以下线为计算证券10日平均收盘价创造了一个因子。

In [2]:
mean_close_10 = SimpleMovingAverage(
    inputs=[USEquityPricing.close],
    window_length=10
)

重要的是要注意创建该因子并不实际执行计算。创建一个因子就像定义函数一样。为了执行计算，我们需要将该因子添加到我们的管道并运行它。

## Adding a Factor to a Pipeline

让我们更新我们原来的空管道，使其计算我们的新移动平均因子。首先，让我们将我们的因子移动至`make_pipeline`中。 接下来，我们可以告诉我们的管道通过传递一个`columns`参数来计算我们的因子，这个列参数应该是一个将列名映射到因子，过滤器或分类器的字典。 我们更新的`make_pipeline`函数应该是这样的：

In [3]:
from zipline.pipeline import Pipeline

In [4]:
def make_pipeline():
    mean_volume_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.volume], window_length=10)
    mean_close_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=10)
    mean_cmv_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.cmv], window_length=10)

    return Pipeline(
        columns={
            '10日平均成交量': mean_volume_10,
            '10_day_mean_close': mean_close_10,
            '10日平均流通市值': mean_cmv_10
        })

为了看看这个样子，让我们制作我们的管道，运行它，并显示结果的前几行。

In [5]:
from zipline.research import run_pipeline

In [6]:
result = run_pipeline(make_pipeline(), '2018-04-10', '2018-04-10')
result.head()

10_day_mean_close     10日平均成交量  \
2018-04-10 00:00:00+00:00 平安银行(000001)             10.985  120813000.0   
                          川化股份(000155)              6.868   26163000.0   
                          美的集团(000333)             54.434   41545000.0   
                          盐湖股份(000792)             12.969   41509000.0   
                          精功科技(002006)              6.371    3322000.0   

                                           10日平均流通市值  
2018-04-10 00:00:00+00:00 平安银行(000001)  1.858443e+11  
                          川化股份(000155)  8.722360e+09  
                          美的集团(000333)  3.520207e+11  
                          盐湖股份(000792)  3.566345e+10  
                          精功科技(002006)  2.899818e+09

现在我们在管道输出中有一列，所有证券的10天平均收盘价格（显示截断）。请注意，每行对应于我们对给定日期存储的给定证券的计算结果。DataFrame有一个`MultiIndex`，其中第一个级别是表示计算日期的日期时间，第二个级别是与证券相对应的`Equity`对象。例如，上面DataFrame中的第一行（2018-04-10 00:00:00+00:00，`平安银行(000001)`包含4月10日平安银行的`mean_close_10`因子结果。

注意：也可以使用`Pipeline.add`方法将因子添加到现有的`Pipeline`实例中。使用添加看起来像这样：

## Latest

In [7]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=10)
    latest_close = USEquityPricing.close.latest

    return Pipeline(columns={
        '10_day_mean_close': mean_close_10,
        'latest_close_price': latest_close
    })

现在，当我们再次制作并运行我们的管线时，我们的输出数据框中有两列。 一列有每个证券的10日平均收盘价，另一列有最新收盘价。

In [8]:
result = run_pipeline(make_pipeline(), '2018-04-10', '2018-04-10')
result.head()

10_day_mean_close  latest_close_price
2018-04-10 00:00:00+00:00 平安银行(000001)             10.985               10.87
                          川化股份(000155)              6.868                7.03
                          美的集团(000333)             54.434               53.01
                          盐湖股份(000792)             12.969               14.03
                          精功科技(002006)              6.371                6.48

## Default Inputs

一些因子具有不应改变的默认输入。例如，`VWAP`内置因子总是通过`USEquityPricing.close`和`USEquityPricing.volume`进行计算。当总是从相同的`BoundColumns`计算出因子时，我们可以在不指定输入的情况下调用构造函数。

In [9]:
from zipline.pipeline.factors import VWAP
vwap = VWAP(inputs=[USEquityPricing.close], window_length=10)